# Python 进阶

## 1. Python 对象的比较、拷贝

In [1]:
a = 10
b = 10
a == b

True

In [2]:
id(a)

1641447647824

In [3]:
id(b)

1641447647824

In [4]:
a is b

True

In [5]:
a = 257
b = 257
a == b

True

In [7]:
id(a)

1641530629008

In [8]:
id(b)

1641530628432

In [6]:
a is b

False

In [9]:

import copy
x = [1]
x.append(x)

y = copy.deepcopy(x)

# 以下命令的输出是？
x == y

RecursionError: maximum recursion depth exceeded in comparison

In [1]:

l = [1, 2, 3]
del l

## 3. 装饰器

In [3]:
def my_decorator(func):
    def wrapper():
        print('wrapper of decorator')
        func()
    return wrapper

def greet():
    print('hello')

greet = my_decorator(greet)
greet()

wrapper of decorator
hello


In [5]:
def my_decorator(func):
    def wrapper():
        print('wrapper of decorator')
        func()
    return wrapper

@my_decorator
def greet():
    print('hello')

greet()

wrapper of decorator
hello


In [6]:
def my_decorator(func):
    def wrapper(message):
        print('wrapper of decorator')
        func(message)
    return wrapper

@my_decorator
def greet(message):
    print(message)

greet('hello')

wrapper of decorator
hello


In [10]:
def my_decorator(func):
    def wrapper(*args, **kwargs):
        print('wrapper of decorator')
        func(*args, **kwargs)
    return wrapper

@my_decorator
def greet(a, b):
    print(a + b)
    print(b)
greet('hello', 'o')

wrapper of decorator
helloo
o


In [12]:
def repeat(num):
    def my_decorator(func):
        def wrapper(*args, **kwargs):
            for i in range(num):
                print('wrapper of decoraotr')
                func(*args, **kwargs)
        return wrapper
    return my_decorator

@repeat(4)
def greet(message):
    print(message)

greet('a')

wrapper of decoraotr
a
wrapper of decoraotr
a
wrapper of decoraotr
a
wrapper of decoraotr
a


In [13]:
greet.__name__

'wrapper'

In [14]:
help(greet)

Help on function wrapper in module __main__:

wrapper(*args, **kwargs)



In [15]:
import functools

def my_decorator(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print('wrapper of decorator')
        func(*args,**kwargs)
    return wrapper

@my_decorator
def greet(message):
    print(message)

greet.__name__

'greet'

In [16]:
class Count:
    def __init__(self, func):
        self.func = func
        self.num_calls = 0

    def __call__(self, *args, **kwargs):
        self.num_calls += 1
        print('num of calls is: {}'.format(self.num_calls))
        return self.func(*args, **kwargs)

@Count
def example():
    print("hello")

example()

num of calls is: 1
hello


In [17]:
example()

num of calls is: 2
hello


In [19]:
import functools

def my_decorator1(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print('execute decorator1')
        func(*args, **kwargs)
    return wrapper

def my_decorator2(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print('execute decorator2')
        func(*args, **kwargs)
    return wrapper

@my_decorator1
@my_decorator2
def greet(message):
    print(message)

greet('hello')

execute decorator1
execute decorator2
hello


### 身份认证

In [ ]:
import functools

def authenticate(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        request = args[0]
        if check_user_logged_in(request):
            return func(*args, **kwargs)
        else:
            raise Exception('Authentication failed')
    return wrapper

@authenticate
def post_comment(request, ...):
    ...

In [ ]:
import time
import functools

def log_execution_time(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        res = func(*args, **kwargs)
        end = time.perf_counter()
        print('{} took {} ms'.format(func.__name__, (end-start)))
        return res
    return wrapper

@log_execution_time
def calculate_similarity(items):
    ...